In [ ]:
%pip install xgboost

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.8/150.0 MB 2.8 MB/s eta 0:00:54
    --------------------------------------- 2.1/150.0 MB 4.5 MB/s eta 0:00:33
    --------------------------------------- 3.1/150.0 MB 5.0 MB/s eta 0:00:30
   - -------------------------------------- 4.2/150.0 MB 4.9 MB/s eta 0:00:30
   - -------------------------------------- 5.0/150.0 MB 5.0 MB/s eta 0:00:30
   - -------------------------------------- 6.3/150.0 MB 5.0 MB/s eta 0:00:29
   - -------------------------------------- 7.1/150.0 MB 4.8 MB/s eta 0:00:30
   -- ------------------------------------- 7.6/150.0 MB 4.6 MB/s eta 0:00:31
   -- ------------------------------------- 8.9/150.0 MB 4.6 MB/s eta 0:00:31
   -- ------------------------------------- 10.2/150.0 MB 4.8 MB/s eta 0:00:30
   -- ------------------------------------- 11.0/150.0 MB 4.8 MB/s eta 0:00:3

In [4]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# ===== Đọc dữ liệu =====
df = pd.read_csv("../data/merge_price_sentiment/FPT_final_training_data.csv")
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# ===== Lọc theo ngày từ 2024-05-16 trở đi =====
start_date = pd.to_datetime("2024-05-16")
df = df[df["Date"] >= start_date]

# ===== Loại bỏ các dòng không có nhãn =====
df = df.dropna(subset=["Target_Label"])

# ===== Chuyển sentiment về kiểu int =====
for col in ["positive", "neutral", "negative", "total_articles"]:
    df[col] = df[col].fillna(0).astype(int)

# ===== Loại bỏ nhãn 0 (giữ nguyên) nếu không muốn dự đoán =====
df = df[df["Target_Label"] != 0]

# ===== Tạo đặc trưng và nhãn =====
features = ["positive", "neutral", "negative", "total_articles"]
X = df[features]
y = df["Target_Label"]

y = y.replace({-1: 0, 1: 1})

# ===== Chia tập huấn luyện và kiểm tra =====
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ===== Huấn luyện mô hình XGBoost =====
model = XGBClassifier(
    objective="binary:logistic",  # Chọn binary:logistic cho nhãn nhị phân
    eval_metric="mlogloss",
    use_label_encoder=False,
    random_state=42
)

model.fit(X_train, y_train)

# ===== Dự đoán và đánh giá =====
y_pred = model.predict(X_test)

print("📋 Classification Report:")
print(classification_report(y_test, y_pred))



📋 Classification Report:
              precision    recall  f1-score   support

         0.0       0.36      0.36      0.36        22
         1.0       0.39      0.39      0.39        23

    accuracy                           0.38        45
   macro avg       0.38      0.38      0.38        45
weighted avg       0.38      0.38      0.38        45



c:\Users\Admin\anaconda3\envs\StockPredict\Lib\site-packages\xgboost\training.py:183: UserWarning: [06:55:31] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
